In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
train_path = Path("../data/processed/train_cleaned.csv").resolve()

out_dir = Path("../data/processed")
out_dir.mkdir(parents=True, exist_ok=True)

train_out_path = out_dir / "train_features.csv"


In [ ]:
train_df = pd.read_csv(train_path)

CabinDeck is included as a candidate feature because deck location may encode passenger location/class information that could relate to survival.

In [ ]:
def create_cabindeck_feat(df):
    df = df.copy()
    df["CabinDeck"] = (
        df["Cabin"]
        .str.extract(r"([A-Za-z])", expand=False)
        .str.upper()
        .fillna("Unknown")
    )
    return df

FamilySize is included as a candidate feature because traveling alone versus with family may capture behavior or access patterns that could relate to survival.

In [ ]:
def create_familysize_feat(df):
    df = df.copy()
    df['FamilySize']=df['SibSp']+df['Parch']+1
    return df

Create a new column to classify the passengers into different age groups

In [ ]:
def age_binning(df):
    # Define age bins and labels
    bins = [0, 12, 19, 39, 59, float("inf")]
    labels = ['Child', 'Teen', 'Adult', 'Middle Aged', 'Senior']

    # Create age_bin column
    df['AgeBin'] = pd.cut(
        df['Age'],
        bins=bins,
        labels=labels,
        include_lowest=True
    )
    return df

In [ ]:
train_df = age_binning(train_df)
train_df = create_cabindeck_feat(train_df)
train_df = create_familysize_feat(train_df)

train_df.head()
train_df.head()

In [ ]:
train_df.to_csv(train_out_path, index=False)
